In [ ]:
import pandas as pd
import dask.dataframe as dd
from datetime import datetime
import dask.bag as db

In [ ]:
#Start Dask client
from dask.distributed import Client

client = Client("tcp://130.37.199.8:35586")
client

In [ ]:
#Start hosting local Wikipedia server
#!prun -v -no-panda ./html-wiki/kiwix-tools/kiwix-serve 1 --port=8989 --threads=32 -v ./html-wiki/data/wikipedia_en_simple_all_nopic_2022-03.zim

In [ ]:
#Load title list of Wikipedia articles
bag = db.read_text('./html-wiki/data/wikipedia_en_simple_all_nopic_2022-03.titles.txt')
bag.count().compute()

In [ ]:
#Scrape articles and extract information to a dataframe
%%time
import requests, io
import pandas as pd
import dask.bag as db
from bs4 import BeautifulSoup

def get_html_tables(titles, host, class_restrict):
    tables = []
    for title in titles:
        title = title.strip()
        try:
            htmlpage = requests.get(f'http://{host}/wikipedia_en_simple_all_nopic_2022-03/A/{title}').text
        except (ValueError, requests.ConnectionError, requests.TooManyRedirects):
            continue
        
        htmlpage = htmlpage.replace('<th>scope="row"</th>', "") # Kiwix
        page = BeautifulSoup(htmlpage, "html.parser")
        for ti, table_node in enumerate(page.find_all("table", list(class_restrict))):
            sectionTitle = table_node.find_previous_sibling("summary")
            sectionTitle = sectionTitle.text.strip() if sectionTitle else ""

            tableCaption = table_node.find("caption")
            tableCaption = tableCaption.text.strip() if tableCaption else ""

            table = {
                'table': str(table_node), 
                'pageTitle':title, 
                'tableIndex': ti,
                'sectionTitle':sectionTitle, 
                'tableCaption':tableCaption
            }
            tables.append(table)
    return tables

host = 'node053:8989'
bag = db.read_text('./html-wiki/data/wikipedia_en_simple_all_nopic_2022-03.titles.txt').repartition(1000)

meta = {'table': str, 'pageTitle':str, 'tableIndex':int, 'sectionTitle':str, 'tableCaption':str}
tables = bag.map_partitions(get_html_tables, host, ['wikitable']).to_dataframe(meta=meta).persist()
print(tables.size.compute())
fout = './html-wiki/data/wikipedia_en_simple_all_nopic_2022-03_tables/'
!rm -r $fout

tables.to_csv(fout + '*.csv.gz', compression='gzip')
!du -h $fout

In [ ]:
#Specify files names and directories
index_wikidata = 'index_simplewiki-20211120.dawg'
lang_version = 'en_simple_all_nopic_2022-03'

directory_dataset = './semtab/'

In [ ]:
#Load dataframe with Wikipedia tables
fout = f'./html-wiki/data/a_wikipedia_{lang_version}_tables/'
!du -h $fout

meta = {'table': str, 'pageTitle':str, 'tableIndex':int, 'sectionTitle':str, 'tableCaption':str, 'articleId': str}
tables = dd.read_csv(fout + '*', compression='gzip', blocksize=None, dtype=meta).persist()#.drop('Unnamed: 0', axis=1)

28M	./html-wiki/data/a_wikipedia_en_simple_all_nopic_2022-03_tables/


In [ ]:
#Matching hyperlinks to Wikidata identifiers is performed using an index based on all redirects and mappings of article titles to Wikidata ID

def get_row_ents(df):
    from bs4 import BeautifulSoup
    import dawg
    lookup = dawg.IntDAWG().load(index_wikidata)
    
    rows = []
    for t in df.itertuples():
        if t.pageTitle not in lookup:
            continue
        soup = BeautifulSoup(t.table, "lxml")
        headers = []
        try:
            headers = [header.text for header in soup.find_all('th')]
        except Exception:
            pass
        for i, tr in enumerate(soup.find_all("tr")):
            ents = []
            ents_names = []
            ents_index = [-1]
            ents_title = ["Page title"]
            for index_cell, td in enumerate(tr.find_all("td")):
                for a in td.find_all("a"):
                    href = a.attrs.get("href")
                    if href and (href[0] != '#') and not href.startswith('http'):
                        if href in lookup:
                            ents.append(lookup[href])
                            ents_names.append(a.text)
                            ents_index.append(index_cell)
                            try:
                                ents_title.append(a['title'])
                            except Exception:
                                ents_title.append("No href title")
            if ents:
                rows.append({
                    'pageHTML': t.table,
                    'pageTitle': t.pageTitle,
                    'articleId': t.articleId,
                    'tableIndex': t.tableIndex,
                    'sectionTitle': t.sectionTitle,
                    'tableCaption': t.tableCaption,
                    'headers': headers,
                    'rowIndex': i,
                    'pageEntity': lookup[t.pageTitle],
                    'rowEntities': ents,
                    'ents_names': ents_names,
                    'ents_index': ents_index,
                    'ents_title': ents_title,
                })
    return pd.DataFrame(rows)
meta = get_row_ents(tables.head(10))
row_ents = tables.map_partitions(get_row_ents, meta=meta).persist()

243065

In [ ]:
#Creating permutations of all pairs of entities included in each row and the page entity.
def add_permutations(df):
    from itertools import permutations
    from bs4 import BeautifulSoup
    rows = []
    for t in df.itertuples():
        wikidata_ids = [t.pageEntity]
        wikidata_names = [str(str(t.pageTitle) + "(pageTitle)")]
        for row_id in t.rowEntities:
            wikidata_ids.append(row_id)
        for wiki_name in t.ents_names:
            wikidata_names.append(wiki_name)
        
        rows.append({
                    'pageHTML': t.pageHTML,
                    'pageTitle': t.pageTitle,
                    'articleId': t.articleId,
                    'tableIndex': t.tableIndex,
                    'sectionTitle': t.sectionTitle,
                    'tableCaption': t.tableCaption,
                    'headers': t.headers,
                    'rowIndex': t.rowIndex,
                    'pageEntity': t.pageEntity,
                    'rowEntities': t.rowEntities,
                    'ents_index': t.ents_index,
                    'ents_title': t.ents_title,
                    'wikidata_names': wikidata_names,
                    'wikidata_ids': wikidata_ids,
                    'pairs': list(permutations(wikidata_ids, 2)),
            
                })
    return pd.DataFrame(rows)
meta = add_permutations(row_ents.head(100))
row_ents_transformed = row_ents.map_partitions(add_permutations, meta=meta).persist().explode('pairs')

20459956

In [ ]:
#Split permutation into rows containing individual pairs
def split_pairs(df):
    from itertools import permutations
    from bs4 import BeautifulSoup
    rows = []
    for t in df.itertuples():
        rows.append({
                    'pageHTML': t.pageHTML,
                    'pageTitle': t.pageTitle,
                    'articleId': t.articleId,
                    'tableIndex': t.tableIndex,
                    'sectionTitle': t.sectionTitle,
                    'tableCaption': t.tableCaption,
                    'headers': t.headers,
                    'rowIndex': t.rowIndex,
                    'pageEntity': t.pageEntity,
                    'rowEntities': t.rowEntities,
                    'ents_index': t.ents_index,
                    'ents_title': t.ents_title,
                    'wikidata_names': t.wikidata_names,
                    'wikidata_ids': t.wikidata_ids,
                    'c1': t.pairs[0],
                    'c2': t.pairs[1],
                })
    return pd.DataFrame(rows)


meta = split_pairs(row_ents_transformed.head(100))
row_ents_transformed = row_ents_transformed.map_partitions(split_pairs, meta=meta).persist()

20459956

In [ ]:
#Load Wikidata statements with qualifiers
df_wd = pd.read_parquet('spoqv.parquet')
df_wd.head()

,s,p,o,pq,v
0,31,38,4916,580,"json:{""time"": {""time"": ""+1999-01-01T00:00:00Z""..."
1,31,38,232415,580,"json:{""time"": {""time"": ""+1830-01-01T00:00:00Z""..."
2,31,38,232415,582,"json:{""time"": {""time"": ""+2002-01-01T00:00:00Z""..."
3,31,2852,25648793,366,"json:{""wikibase-item"": 6498663}"
4,31,2852,25648794,366,"json:{""wikibase-item"": 35535}"


In [ ]:
#Database-style join, keys that we use are pair of identifiers from the Wikipedia table and subject and object from Wikidata
row_ents_transformed = row_ents_transformed.merge(df_wd, left_on=['c1','c2'], right_on=['s','o'])

In [ ]:
def find_match_wiki_item(c1, c2, wikibase_item, wikidata_ids, ents_index):
    try:
        c1 = int(c1)
        c2 = int(c2)
        wikibase_item = int(wikibase_item)
        if wikibase_item == c1 or wikibase_item == c2:
            return 0
        else:
            match = 0
            for i, wikidata_id in enumerate(wikidata_ids):
                if int(wikidata_id) == wikibase_item:
                    match = 0
                    try:
                        if ents_index[wikidata_ids.index(int(c1))] != ents_index[wikidata_ids.index(int(c2))]:
                            return 1
                    except Exception:
                        pass
            return match
    except Exception:
        return 0
    
#Find matches based on qualifier value
def extract_qualifier_value(df):
    from itertools import permutations
    from bs4 import BeautifulSoup
    import json
    rows = []
    for t in df.itertuples():
        qv_match = 0
        year_match = 0
        year_match_str = 0
        wikibase_item = -1
        year = -1
        wikidata_names = t.wikidata_names
        wikidata_ids =  t.wikidata_ids
        
        try:
            wikibase_item = json.loads(t.v.replace("json:",""))['wikibase-item']
            qv_match = find_match_wiki_item(t.c1, t.c2, wikibase_item, t.wikidata_ids, t.ents_index)
        except Exception:
            pass
        try:
            year = t.v.split("+")[1].split("-")[0]
            for cell in t.wikidata_names:
                if str(year) == str(cell):
                    wikidata_ids.append(-9)
                    wikidata_names.append(str(year)+"(cell_text)")
                    year_match = 1
                    break
        except Exception:
            pass
        
        try:
            year = t.v.split("+")[1].split("-")[0]
            for cell in t.wikidata_names:
                if str(cell).find(str(year)) > -1:
                    wikidata_ids.append(-9)
                    wikidata_names.append(str(year)+"(cell_text_str)")
                    year_match_str = 1
                    break
        except Exception:
            pass
        rows.append({
                    'pageHTML': t.pageHTML,
                    'pageTitle': t.pageTitle,
                    'articleId': t.articleId,
                    'tableIndex': t.tableIndex,
                    'sectionTitle': t.sectionTitle,
                    'tableCaption': t.tableCaption,
                    'headers': t.headers,
                    'rowIndex': t.rowIndex,
                    'pageEntity': t.pageEntity,
                    'rowEntities': t.rowEntities,
                    'ents_index': t.ents_index,
                    'ents_title': t.ents_title,
                    'wikidata_names': wikidata_names,
                    'wikidata_ids': wikidata_ids,
                    'c1': t.c1,
                    'c2': t.c2,
                    's': t.s,
                    'p': t.p,
                    'o': t.o,
                    'pq': t.pq,
                    'v': t.v,
                    'wikibase_item': wikibase_item,
                    'year': year,
                    'qv_match': qv_match,
                    'year_match': year_match,
                    'year_match_str': year_match_str,
                })
    return pd.DataFrame(rows)


meta = extract_qualifier_value(row_ents_transformed.head(100))
row_ents_transformed = row_ents_transformed.map_partitions(extract_qualifier_value, meta=meta).persist()

In [ ]:
#Save into pandas dataframe
r = row_ents_transformed.compute()
r = r.drop(['pageTitle'], axis=1)
r = r.rename(columns={"articleId": "pageTitle", "pageHTML": "HTML"})

In [ ]:
#Retrieve only n-ary tables
def is_nary(m, m2, m3):
    try:
        if int(m) > 0 or  int(m2) > 0 or  int(m3) > 0:
            return True
        return False
    except:
        return False
r['is_nary'] = r.apply(lambda row : is_nary(row['qv_match'], row['year_match'],row['year_match_str']), axis = 1)
r_only_nary = r[r.is_nary == True]
r_only_nary['lang'] = lang_version
r_only_nary

/tmp/ipykernel_19753/2487779041.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r_only_nary['lang'] = 'en_simple_2022-03'


,HTML,pageTitle,tableIndex,sectionTitle,tableCaption,headers,rowIndex,pageEntity,rowEntities,ents_index,...,o,pq,v,wikibase_item,year,qv_match,year_match,year_match_str,is_nary,lang
88,"<table class=""wikitable succession-box"" style=...",Giorgi_Kvirikashvili,0,Other websites,NaN,[Political offices\n],3,15277881,"[6066533, 1975365, 54982996]","[-1, 0, 1, 2]",...,1975365,1366,"json:{""wikibase-item"": 54982996}",54982996,-1,1,0,0,True,en_simple_2022-03
89,"<table class=""wikitable succession-box"" style=...",Giorgi_Kvirikashvili,0,Other websites,NaN,[Political offices\n],3,15277881,"[6066533, 1975365, 54982996]","[-1, 0, 1, 2]",...,1975365,1365,"json:{""wikibase-item"": 6066533}",6066533,-1,1,0,0,True,en_simple_2022-03
92,"<table class=""wikitable succession-box"" style=...",Giorgi_Kvirikashvili,0,Other websites,NaN,[Political offices\n],3,15277881,"[6066533, 1975365, 54982996]","[-1, 0, 1, 2]",...,1975365,1366,"json:{""wikibase-item"": 15277881}",15277881,-1,1,0,0,True,en_simple_2022-03
98,"<table class=""wikitable succession-box"" style=...",Giorgi_Kvirikashvili,0,Other websites,NaN,[Political offices\n],3,15277881,"[6066533, 1975365, 54982996]","[-1, 0, 1, 2]",...,1975365,1365,"json:{""wikibase-item"": 15277881}",15277881,-1,1,0,0,True,en_simple_2022-03
15,"<table border=""1"" cellpadding=""3"" cellspacing=...",1960–61_NHL_season,3,NHL awards,NaN,[1960–61 NHL awards],1,1960705,"[1133502, 188143]","[-1, 0, 1]",...,1133502,585,"json:{""time"": {""time"": ""+1960-01-01T00:00:00Z""...",-1,1960,0,0,1,True,en_simple_2022-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,"<table class=""wikitable"" style=""text-align:cen...",2019–20_FC_Bayern_Munich_season,2,Transfers in,NaN,"[Date from, Position, Player, Transferred from...",1,64476747,"[142, 19599124, 154191, 4512]","[-1, 2, 2, 3, 3]",...,4512,582,"json:{""time"": {""time"": ""+2019-06-30T00:00:00Z""...",-1,2019,0,0,1,True,en_simple_2022-03
100,"<table class=""wikitable succession-box"" style=...",Martin_H._Kennelly,0,References,NaN,[Political offices\n],2,492211,"[491933, 1760078, 295267]","[-1, 0, 1, 2]",...,1760078,1365,"json:{""wikibase-item"": 491933}",491933,-1,1,0,0,True,en_simple_2022-03
101,"<table class=""wikitable succession-box"" style=...",Martin_H._Kennelly,0,References,NaN,[Political offices\n],2,492211,"[491933, 1760078, 295267]","[-1, 0, 1, 2]",...,1760078,1366,"json:{""wikibase-item"": 295267}",295267,-1,1,0,0,True,en_simple_2022-03
105,"<table class=""wikitable succession-box"" style=...",Martin_H._Kennelly,0,References,NaN,[Political offices\n],2,492211,"[491933, 1760078, 295267]","[-1, 0, 1, 2]",...,1760078,1366,"json:{""wikibase-item"": 492211}",492211,-1,1,0,0,True,en_simple_2022-03


In [ ]:
#Group matches into individual tables, and get a random row where a match was located
r_only_nary_grouped = r_only_nary.groupby(['pageTitle', 'tableIndex']).apply(lambda x: x.sample(1)).reset_index(drop=True)
r_only_nary_grouped

,HTML,pageTitle,tableIndex,sectionTitle,tableCaption,headers,rowIndex,pageEntity,rowEntities,ents_index,...,o,pq,v,wikibase_item,year,qv_match,year_match,year_match_str,is_nary,lang
0,"<table class=""wikitable"">\n<tbody><tr>\n<th>Ac...",102_Dalmatians,0,Cast,NaN,"[Actor, Character, Films]",1,165847,"[372311, 388605]","[-1, 0, 1]",...,372311,642,"json:{""wikibase-item"": 165847}",165847,-1,1,0,0,True,en_simple_2022-03
1,"<table border=""1"" cellpadding=""3"" cellspacing=...",1927–28_NHL_season,4,NHL awards,NaN,[1927–28 NHL awards],5,939682,"[221401, 1354292, 188143]","[-1, 0, 1, 1]",...,221401,585,"json:{""time"": {""time"": ""+1927-01-01T00:00:00Z""...",-1,1927,0,0,1,True,en_simple_2022-03
2,"<table border=""1"" cellpadding=""3"" cellspacing=...",1928–29_NHL_season,3,NHL awards,NaN,[1928–29 NHL awards],5,239929,"[221401, 1354292, 188143]","[-1, 0, 1, 1]",...,221401,585,"json:{""time"": {""time"": ""+1928-01-01T00:00:00Z""...",-1,1928,0,0,1,True,en_simple_2022-03
3,"<table border=""1"" cellpadding=""3"" cellspacing=...",1929–30_NHL_season,3,NHL awards,NaN,[1929–30 NHL awards],4,761919,"[516636, 1354316, 188984]","[-1, 0, 1, 1]",...,516636,585,"json:{""time"": {""time"": ""+1929-01-01T00:00:00Z""...",-1,1929,0,0,1,True,en_simple_2022-03
4,"<table border=""1"" cellpadding=""3"" cellspacing=...",1930–31_NHL_season,4,NHL awards,NaN,[1930–31 NHL awards],4,939658,"[516636, 1354316, 188984]","[-1, 0, 1, 1]",...,516636,585,"json:{""time"": {""time"": ""+1930-01-01T00:00:00Z""...",-1,1930,0,0,1,True,en_simple_2022-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,"<table class=""wikitable succession-box"" style=...",Zurab_Noghaideli,0,Other websites,NaN,[],1,316238,"[312493, 1975365, 337564]","[-1, 0, 1, 2]",...,1975365,1366,"json:{""wikibase-item"": 316238}",316238,-1,1,0,0,True,en_simple_2022-03
960,"<table class=""wikitable succession-box"" style=...",Éamon_de_Valera,0,Other websites,NaN,[Parliament of the United Kingdom (1801–presen...,43,173347,"[381690, 191827]","[-1, 0, 1]",...,191827,1365,"json:{""wikibase-item"": 173347}",173347,-1,1,0,0,True,en_simple_2022-03
961,"<table class=""wikitable"" style=""text-align:rig...",Ñuble_Province_(1974-2018),0,NaN,Ñuble province,"[Code, Comuna, Capital, Area[4](km²), Populati...",14,721755,[56115],"[-1, 2]",...,721755,582,"json:{""time"": {""time"": ""+2018-09-05T00:00:00Z""...",-1,2018,0,0,1,True,en_simple_2022-03
962,"<table class=""wikitable succession-box"" style=...",Östen_Undén,0,References,NaN,[Political offices\n],8,298317,"[53660, 687075, 53707]","[-1, 0, 1, 2]",...,687075,1365,"json:{""wikibase-item"": 53660}",53660,-1,1,0,0,True,en_simple_2022-03


In [ ]:
#Save to CSV
date = str(datetime.now()).replace(":", "").replace(" ","_")
r_only_nary_grouped[['lang','pageTitle', 'tableIndex','pageEntity','sectionTitle','tableCaption', 'headers','HTML']] \
.to_csv(f"{directory_dataset}{date}_{lang_version}_tables.csv") 
r_only_nary[['lang','pageTitle', 'tableIndex', 'rowIndex','wikidata_ids','ents_index', 'ents_title' ,'wikidata_names', 's','p','o', 'pq', 'v', 'qv_match','year_match','year_match_str']] \
.to_csv(f"{directory_dataset}{date}_{lang_version}_matches.csv")